# Lion Cheetah Classification task

Baed on the following Kaggle data set: https://www.kaggle.com/datasets/mikoajfish99/lions-or-cheetahs-image-classification

In [ ]:
# import essential packages

import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf